# Pure State Tomography Experiments
This notebook contains code for running experiments on IBM quantum computers.

## First Time Setup
Before running the code in this repository, you will need to set up the IBM Qiskit Runtime. First, create a file named config.ini, which will contain your IBM credentials. The format is as follows:

```ini
[IBM]
token = <YOUR_IBM_TOKEN>
```

This program will read the config.ini file for your API token. Now, run the following code to save your IBM token to your machine for further use. You will no longer need to run this cell afterwards. 

In [1]:
# import configparser
# from qiskit_ibm_runtime import QiskitRuntimeService

# with open("config.ini", "r") as cf:
#     cp = configparser.ConfigParser()
#     cp.read_file(cf)
#     api_token = cp.get("IBM", "token")
# QiskitRuntimeService.save_account(channel="ibm_quantum", token=api_token, overwrite=True)

You will also need to install the prerequisite python packages.

### Running Experiments
First, import the necessary packages and set up the code:

In [1]:
from numpy import (
    ndarray,
    asarray,
    array,
    reshape,
    sqrt,
    linalg,
    set_printoptions
)
from qiskit_aer import AerSimulator
from pure_state_tomography import tomography
import putils
import qutils
import qiskit
from measurement_manager import measurement_manager

set_printoptions(linewidth=100)

The following function runs the experiment using the `pure_state_tomography` package. 

For runs on real IBM quantum computers (i.e. not simulator or statevector runs), the first concluded run for an input will output a job file containing the IBM job ids, which the `tomography` class will use to recall the contents of previous runs. Do note that because concurrent IBM jobs are limited to 3 per free account, you may have to run this code several times to get all of the needed measurement results.  

In [2]:
talg = tomography()

def run(
    mm: measurement_manager,
    state: ndarray | qiskit.QuantumCircuit,
    verbose: bool = True,
    job_file: str = None,
    hadamard: bool = False,
):
    putils.fprint("Running inference at {} shots\n".format(mm.n_shots))

    # reset measurement manager state
    mm.set_state(state)
    
    if type(state) is qiskit.QuantumCircuit:
        state = state.copy()
    
    # run pure state tomography
    res = talg.pure_state_tomography(
        mm=mm, verbose=verbose, job_file=job_file, hadamard=hadamard
    )

    if res is not None:        
        if type(state) is ndarray:
            if state.ndim > 1:
                res = (
                    reshape(
                        res,
                        (
                            state.shape[0],
                            state.shape[0],
                        ),
                    ).T
                    * 2
                )
            putils.fprint(
                "Reconstructed {}:\n{}".format(
                    "vector" if state.ndim == 1 else "matrix", res
                )
            )
            putils.fprint(
                "% Error: {}\n".format(100 * linalg.norm(state- res))
            )
        elif type(state) is qiskit.QuantumCircuit:
            state = qutils.circuit_to_statevector(state)
            
            if state.ndim > 1:
                res = (
                    reshape(
                        res,
                        (
                            state.shape[0],
                            state.shape[0],
                        ),
                    ).T
                    * 2
                )

            putils.fprint(
                "Original {}:\n{}".format(
                    "vector" if state.ndim == 1 else "matrix", state
                )
            )
            putils.fprint(
                "Reconstructed {}:\n{}".format(
                    "vector" if state.ndim == 1 else "matrix", res
                )
            )

            putils.fprint(
                "% Error: {}\n".format(100 * linalg.norm(state- res))
            )

The following code runs the experiment. Using a single measurement manager object is recommended. 

In [4]:
VERBOSITY = True
# qutils.EPSILON = 1.5e-2  # matrix
qutils.EPSILON = 5e-3  # vector
# qutils.EPSILON = 5e-5  # statevector

"""
needed experiments:
000, 001            job_2023_11_11T_02_41_28.txt
state = qiskit.QuantumCircuit(3)
state.h(0)

000, 110            job_2023_11_11T_02_42_01.txt
state = qiskit.QuantumCircuit(3)
state.h(2)
state.x(2)
state.cnot(2, 1)
           
000, 111            job_2023_11_11T_02_43_16.txt
state = qiskit.QuantumCircuit(3)
state.h(2)
state.x(2)
state.cnot(2, 1)
state.cnot(2, 0)
  
000, 001, 010,      
000, 001, 110       
000, 011, 101       
general unitary

HADAMARD
000, 001            job_2023_11_12T_15_20_57.txt
state = qiskit.QuantumCircuit(3)
state.h(0)

000, 110            job_2023_11_12T_15_21_26.txt
state = qiskit.QuantumCircuit(3)
state.h(2)
state.x(2)
state.cnot(2, 1)
           
000, 111            
state = qiskit.QuantumCircuit(3)
state.h(2)
state.x(2)
state.cnot(2, 1)
state.cnot(2, 0)
  
000, 001, 010,      
000, 001, 110       
000, 011, 101  
"""

JOB_FILE = "job_2023_11_12T_15_21_26.txt"

VERBOSITY = True

# put state code here
state = qiskit.QuantumCircuit(3)
state.h(2)
state.x(2)
state.cnot(2, 1)

mm = measurement_manager(
    n_shots=16384,
    execution_type=qutils.execution_type.ibm_qpu,
    verbose=VERBOSITY,
)

run(mm=mm, state=state, verbose=VERBOSITY, job_file=JOB_FILE, hadamard=True)

Running inference at 16384 shots

Input circuit:
                    
q_0: ───────────────
               ┌───┐
q_1: ──────────┤ X ├
     ┌───┐┌───┐└─┬─┘
q_2: ┤ H ├┤ X ├──■──
     └───┘└───┘     
Statevector:
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j]
                     ░ ┌───┐ ░  ░ ┌───┐
q_0: ────────────────░─┤ H ├─░──░─┤ I ├
               ┌───┐ ░ ├───┤ ░  ░ ├───┤
q_1: ──────────┤ X ├─░─┤ H ├─░──░─┤ I ├
     ┌───┐┌───┐└─┬─┘ ░ ├───┤ ░  ░ ├───┤
q_2: ┤ H ├┤ X ├──■───░─┤ H ├─░──░─┤ I ├
     └───┘└───┘      ░ └───┘ ░  ░ └───┘
Dry run measurements:
Circuits for source index 0 and target index 1:
                     ░ ┌───┐ ░ ┌─────────┐
q_0: ────────────────░─┤ H ├─░─┤ Unitary ├
               ┌───┐ ░ ├───┤ ░ └──┬───┬──┘
q_1: ──────────┤ X ├─░─┤ H ├─░────┤ I ├───
     ┌───┐┌───┐└─┬─┘ ░ ├───┤ ░    ├───┤   
q_2: ┤ H ├┤ X ├──■───░─┤ H ├─░────┤ I ├───
     └───┘└───┘      ░ └───┘ ░    └───┘   
                 

IBMBackendApiError: 'Error submitting job: \'409 Client Error: Conflict for url: https://api.quantum-computing.ibm.com/runtime/jobs. {"errors":[{"message":"You have reached the limit of 3 pending  jobs. Please wait for a job to complete or cancel one before submitting anything new.","code":3458,"solution":"Wait until some previous jobs were finished. You can cancel pending jobs to run new jobs.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''

In [3]:
%%capture
VERBOSITY = True
mm = measurement_manager(
    n_shots=16384,
    execution_type=qutils.execution_type.simulator,
    verbose=VERBOSITY,
)
# for a in range(500):
    # put state code here
state = qiskit.QuantumCircuit(3)
state.h(2)
state.x(2)
state.cnot(2, 1)
run(mm=mm, state=state, verbose=VERBOSITY, hadamard=True)

Running inference at 16384 shots

Input circuit:
                    
q_0: ───────────────
               ┌───┐
q_1: ──────────┤ X ├
     ┌───┐┌───┐└─┬─┘
q_2: ┤ H ├┤ X ├──■──
     └───┘└───┘     
Statevector:
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j]
                        ░ ┌───┐ ░ ┌───┐ ░ ┌─┐      
   q_0: ────────────────░─┤ H ├─░─┤ I ├─░─┤M├──────
                  ┌───┐ ░ ├───┤ ░ ├───┤ ░ └╥┘┌─┐   
   q_1: ──────────┤ X ├─░─┤ H ├─░─┤ I ├─░──╫─┤M├───
        ┌───┐┌───┐└─┬─┘ ░ ├───┤ ░ ├───┤ ░  ║ └╥┘┌─┐
   q_2: ┤ H ├┤ X ├──■───░─┤ H ├─░─┤ I ├─░──╫──╫─┤M├
        └───┘└───┘      ░ └───┘ ░ └───┘ ░  ║  ║ └╥┘
meas: 3/═══════════════════════════════════╩══╩══╩═
                                           0  1  2 
Measurements:
Circuits for source index 0 and target index 1:
                        ░ ┌───┐ ░ ┌─────────┐ ░ ┌─┐      
   q_0: ────────────────░─┤ H ├─░─┤ Unitary ├─░─┤M├──────
                  ┌───